In [6]:
import pandas as pd
import numpy as np

url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz'
url_data = pd.read_table(url, sep='\t', nrows=200000, error_bad_lines=False)

b'Skipping line 9076: expected 15 fields, saw 22\nSkipping line 19256: expected 15 fields, saw 22\nSkipping line 24313: expected 15 fields, saw 22\nSkipping line 47211: expected 15 fields, saw 22\nSkipping line 54295: expected 15 fields, saw 22\nSkipping line 56641: expected 15 fields, saw 22\nSkipping line 63067: expected 15 fields, saw 22\n'
b'Skipping line 93796: expected 15 fields, saw 22\n'
b'Skipping line 132806: expected 15 fields, saw 22\nSkipping line 164631: expected 15 fields, saw 22\nSkipping line 167019: expected 15 fields, saw 22\nSkipping line 167212: expected 15 fields, saw 22\n'
b'Skipping line 198103: expected 15 fields, saw 22\nSkipping line 199191: expected 15 fields, saw 22\n'


In [7]:
url_data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,41409413,R2MTG1GCZLR2DK,B00428R89M,112201306,yoomall 5M Antenna WIFI RP-SMA Female to Male ...,Electronics,5,0,0,N,Y,Five Stars,As described.,2015-08-31
1,US,49668221,R2HBOEM8LE9928,B000068O48,734576678,"Hosa GPM-103 3.5mm TRS to 1/4"" TRS Adaptor",Electronics,5,0,0,N,Y,It works as advertising.,It works as advertising.,2015-08-31
2,US,12338275,R1P4RW1R9FDPEE,B000GGKOG8,614448099,Channel Master Titan 2 Antenna Preamplifier,Electronics,5,1,1,N,Y,Five Stars,Works pissa,2015-08-31
3,US,38487968,R1EBPM82ENI67M,B000NU4OTA,72265257,LIMTECH Wall charger + USB Hotsync & Charging ...,Electronics,1,0,0,N,Y,One Star,Did not work at all.,2015-08-31
4,US,23732619,R372S58V6D11AT,B00JOQIO6S,308169188,Skullcandy Air Raid Portable Bluetooth Speaker,Electronics,5,1,1,N,Y,Overall pleased with the item,Works well. Bass is somewhat lacking but is pr...,2015-08-31


In [21]:
url_product = url_data.set_index('product_title')
by_ptitle = url_product.groupby('product_title')['star_rating'].agg(['mean', 'count'])
by_ptitle.columns = ['Avg Rating', '# Of Reviews']

In [22]:
def weighted_rank(df):
    total_count = by_ptitle['# Of Reviews'].count()
    threshold = by_ptitle['# Of Reviews'].mean() + by_ptitle['# Of Reviews'].std()
    by_ptitle['count_rank'] = by_ptitle['# Of Reviews'].rank(ascending=False)
    by_ptitle['rating_rank'] = by_ptitle['Avg Rating'].rank(ascending=False)
    by_ptitle['rank_score'] = np.where(by_ptitle['# Of Reviews'] >= threshold,
                                       by_ptitle['Avg Rating']/5 + (total_count-by_ptitle['count_rank'])/total_count,
                                       (by_ptitle['Avg Rating']/5 + ((1-((threshold-by_ptitle['# Of Reviews'])/threshold))*(total_count-by_ptitle['count_rank'])/total_count)))
    by_ptitle['rank'] = by_ptitle['rank_score'].rank(ascending=False)

In [24]:
weighted_rank(by_ptitle)
clean_df = by_ptitle[['Avg Rating', '# Of Reviews', 'rank']].sort_values('rank', ascending=True)
clean_df

,Avg Rating,# Of Reviews,rank
product_title,,,
"Swimbuds SPORT Waterproof Headphones - See below under ""Special Offers and Product Promotions"" for discounts on this Headphone",4.925000,120,1.0
CITI ULTRA Series - PREMIUM HDMI CABLE HIGH SPEED 50 FEET [15 M] Black Nylon Brading with 24k Gold Plated,5.000000,40,2.0
Caseling Premium Hard EVA Case Travel Bag Pouch for Bose Soundlink Mini Bluetooth Speaker,4.891473,387,3.0
CITI - PREMIUM 25FT 3.5mm Stereo Male to 2RCA Male Audio AUX Cable 25 FEET - Gold Plated [ iPhone iPad iPod MP3 Smartphones],5.000000,35,4.0
"Etekcity 10 Pack Power Extension Cord Cable, Outlet Saver, 3 Prong, 16AWG-13A, UL Listed (Black, 1-Foot)",4.897727,88,5.0
"Sewell Direct SW-29863-12 Deadbolt Banana Plugs, 12-Pair",4.871345,171,6.0
Underwater Audio Waterproof iPod Shuffle,4.886598,97,7.0
"Cable Management Sleeve, JOTO Cable Sleeve, [Set of 4] 19"" Neoprene Cable Sleeves for PC / TV / Home Theater / Speaker, Flexible Cable Wrap, Cable Cover, Cable Organizer, Cords Management (4 Piece)",4.870504,139,8.0
"iDeaUSA Dual Bluetooth Wireless Speakers - Portable Speaker Combo Pack, 4x Woofers with Buddy Setup, Boombox, Up to 10 hours play time, Built in Mic, NFC Compatible for iPhone, Samsung, Nexus and more",4.924528,53,9.0


In [25]:
clean_df.describe()

,Avg Rating,# Of Reviews,rank
count,35230.000000,35230.000000,35230.000000
mean,4.018984,5.676980,17615.500000
std,1.239650,26.633817,9990.550791
min,1.000000,1.000000,1.000000
25%,3.500000,1.000000,13940.500000
50%,4.500000,1.000000,13940.500000
75%,5.000000,3.000000,25208.000000
max,5.000000,2274.000000,33897.500000
